In [4]:
import coffea
import numpy as np
import awkward as ak
from coffea import hist, processor
from coffea.analysis_tools import PackedSelection
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

In [5]:
fname = '/afs/crc.nd.edu/user/i/imcalist/Public/XConettbar.root'
events = NanoEventsFactory.from_root(fname, schemaclass=NanoAODSchema,entry_start=0).events()

In [2]:
def isPresElec(pt, eta, dxy, dz, miniIso):
    pt    = (pt       > 32)
    eta   = (abs(eta) < 2.4)
    dxy   = (abs(dxy) < 0.05)
    dz    = (abs(dz)  < 0.1)
    iso   = (miniIso  < 0.4)
    return (pt & eta & dxy & dz & iso)

def isPresMuon(dxy, dz, eta, pt):
    pt_mask    = (pt         > 27)
    eta_mask   = (abs(eta)   < 2.4)
    dxy_mask   = (abs(dxy)   < 0.2)
    dz_mask    = (abs(dz)    < 0.5)
    return (pt_mask & eta_mask & dxy_mask & dz_mask)

def isLooseElec(miniPFRelIso_all,sip3d,lostHits):
    return (miniPFRelIso_all<0.4) & (sip3d<8) & (lostHits<=1)

def isLooseMuon(miniPFRelIso_all,sip3d):
    return (miniPFRelIso_all<0.4) & (sip3d<8)

def isClean(obj_A, obj_B, drmin=0.4):
    objB_near, objB_DR = obj_A.nearest(obj_B, return_metric=True)
    mask = ak.fill_none(objB_DR > drmin, True)
    return (mask)

In [6]:
e = events.Electron
j = events.XConeJet
j = ak.with_name(j, "PtEtaPhiMLorentzVector")
mu = events.Muon
met = events.MET
print("Total events: "+ str(ak.num(e, axis=0)))

Total events: 14400


In [8]:
e['isPresElec'] = isPresElec(e.pt, e.eta, e.dxy, e.dz, e.miniPFRelIso_all)
e['isLooseElec'] = isLooseElec(e.miniPFRelIso_all, e.sip3d, e.lostHits)
e = e[e.isPresElec & e.isLooseElec]
elecpt = ak.flatten(e.pt)
eleceta = ak.flatten(e.eta)
elecphi = ak.flatten(e.phi)
elecCount = ak.num(e, axis=1)

In [9]:
mu['isPresMuon']= isPresMuon(mu.dxy, mu.dz, mu.eta, mu.pt)
mu['isLooseMuon'] = isLooseMuon(mu.miniPFRelIso_all, mu.sip3d)
mu = mu[mu.isPresMuon & mu.isLooseMuon]
muonpt = ak.flatten(mu.pt)
muoneta = ak.flatten(mu.eta)
muonphi = ak.flatten(mu.phi)
muonCount = ak.num(mu,axis=1)

In [10]:
singLep = elecCount + muonCount ==1

In [13]:
met = events.MET
met.pt

<Array [23.6, 92.5, 75.2, ... 26.5, 95.9, 23.4] type='14400 * float32[parameters...'>

In [15]:
met = met[singLep]
miss = ak.flatten(met.pt, axis = 0)
miss

<Array [21.6, 15.8, 173, ... 84.8, 26.5, 23.4] type='6232 * float32[parameters={...'>